# **Análise Exploratória de Dados de Logística**

## 1\. Contexto

A intenção por trás desse notebook é fazer uma análise exploratória de dados utilizando como base de dados o dataset da empresa Loggi, uma empresa grande de logística aqui do Brasil.

Aqui faremos a exploração e uma breve manipulação dos dados para que seja possivel transformar um arquivo json em toda sua complexidade em um DataFrame pandas. 

## 2\. Pacotes e bibliotecas

In [ ]:
!pip install wget
!pip3 install geopandas
!pip install geopy

In [ ]:
import json

import wget
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np
import geopandas
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile

## 3\. Exploração de dados

In [ ]:
# Importação do arquivo .json da Loggi
wget.download("https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json")
with open('deliveries.json', mode='r', encoding='utf8') as file:
  data = json.load(file)

Na sequência, o próximo passo envolve  verificar a composição desse arquivo json, agora salvo na variavel data. Para isso, foi pego a primeira linha de cada coluna como amostra para melhor compreensão.

In [ ]:
len(data)

In [ ]:
example = data[0]

In [ ]:
example['name']

In [ ]:
example['region']

In [ ]:
example['origin']

In [ ]:
example['origin']['lat']

In [ ]:
example['vehicle_capacity']

In [ ]:
example['deliveries'][0]['point']['lng']

Sabendo como está configurado o arquivo, transformaremos essa variavel data em um dataframe pandas.

In [ ]:
deliveries_df = pd.DataFrame(data)
deliveries_df

A primeira coluna a ser verificada foi a origin, pois ela ainda está em json.

Então será executado uma sequência de comandos para que as informações de Latitude e Longitude do Hub, padrão universal de geolocalização, fiquem dispostas como dataframe pandas.

In [ ]:
# Esse comando transforma a coluna 'origin' em um dataframe pandas.
hub_origin_df = pd.json_normalize(deliveries_df["origin"])

# Comando para juntar o dataframe original 'deliveries_df' com o criado no passo anterior.
deliveries_df = pd.merge(left=deliveries_df, right=hub_origin_df, how='inner', left_index=True, right_index=True)

# Como a coluna 'origin' está em json e seus valores foram somados aos do dataframe original, é preciso deletar o 'origin' em json.
deliveries_df = deliveries_df.drop("origin", axis=1)

# Reconfigurando o dataframe para a ordem e de disposição e nome das colunas de interesse.
deliveries_df = deliveries_df[["name", "region", "lng", "lat", "vehicle_capacity", "deliveries"]]
deliveries_df.rename(columns={"lng": "hub_lng", "lat": "hub_lat"}, inplace=True)

Outra coluna que precisa ser padronizada no padrão do dataframe é a 'deliveries'. Como ela envolve uma lista, pois é referente as entregas de um único chamado e, por isso, possui mais de 1 linha, será preciso quebrar a lista e retirar as informações necessárias.

In [ ]:
# O comando explodes retira os elementos da lista, transformam eles em um dataframe.
deliveries_exploded_df = deliveries_df[["deliveries"]].explode("deliveries")

In [ ]:
# Agora será criado um novo dataframe, apenas com os valores de interesse retirados do deliveries_exploded_df (tamanho, latitude e longitude do local de entrega). 
deliveries_normalized_df = pd.concat([
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda x: x["size"])).rename(columns={"deliveries": "delivery_size"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda x: x["point"]["lng"])).rename(columns={"deliveries": "delivery_lng"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda x: x["point"]["lat"])).rename(columns={"deliveries": "delivery_lat"}),
], axis= 1)


In [ ]:
# Então será removida a coluna 'deliveries' do dataframe 'deliveries_df' e incluido as colunas do 'deliveries_normalized_df'.
deliveries_df = deliveries_df.drop("deliveries", axis=1)
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_normalized_df, how='right', left_index=True, right_index=True)

# Comando para resetar o index.
deliveries_df.reset_index(inplace=True, drop=True)
deliveries_df.head()

Sobre a estrutura do dataframe:

In [ ]:
# Tamanho do dataframe, linhas por colunas
deliveries_df.shape

In [ ]:
# Informações ssobre os tipos de dados trabalhados e quantos valores nulos em cada coluna
deliveries_df.info()

In [ ]:
# Considerações sobre os valores em objeto do dataframe
deliveries_df.select_dtypes("object").describe().transpose()

In [ ]:
# Informações numéricas das colunas com valores em int, como a média e os quartis.
deliveries_df.drop(["name", "region"], axis=1).select_dtypes('int64').describe().transpose()

In [ ]:
# Checagem para saber se há algum valor nulo não listado anteriormente
deliveries_df.isna().any()

## 4\. Manipulação


Como a localização dos hubs e entregas está em Latitude e longitude, é possivel utilizar esses valores para encontrar outras informações, como bairro e cidade. Foi utilizado o geopy para as operações.

In [ ]:
# Primeiro é preciso separar os dados de latitude e longitude do hub
hub_df = deliveries_df[["region", "hub_lng", "hub_lat"]]
hub_df = hub_df.drop_duplicates().sort_values(by="region").reset_index(drop=True)
hub_df.head()

In [ ]:
# Nominatim é uma open-source de OpenStreetMap data gratuito, porém por 1 solicitação por segundo
geolocator = Nominatim(user_agent="ebac_geocoder")
geocoder = RateLimiter(geolocator.reverse, min_delay_seconds=1)

In [ ]:
# Concatenando as colunas 'hub_lat' e 'lub_lng' para que a variável geocoder possa encontrar as informações requeridas
hub_df["coordinates"] = hub_df["hub_lat"].astype(str)  + ", " + hub_df["hub_lng"].astype(str) 
hub_df["geodata"] = hub_df["coordinates"].apply(geocoder)

In [ ]:
# Novo dataframe apenas com as informações presentes na coluna 'geodata' do hub_df, normalizando os dados
hub_geodata_df = pd.json_normalize(hub_df["geodata"].apply(lambda data: data.raw))
hub_geodata_df.head()

In [ ]:
# Seleção das colunas de interesse e correção dos nomes das colunas para melhor vizualização
hub_geodata_df = hub_geodata_df[["address.town", "address.suburb", "address.city"]]
hub_geodata_df.rename(columns={"address.town": "hub_town", "address.suburb": "hub_suburb", "address.city": "hub_city"}, inplace=True)

# para hub_geodata_df["hub_suburb"], se existir valor na coluna 'hub_city' mantém o valor. Senão, considere 'hub_town'
hub_geodata_df["hub_city"] = np.where(hub_geodata_df["hub_city"].notna(), hub_geodata_df["hub_city"], hub_geodata_df["hub_town"])
# para hub_geodata_df["hub_suburb"], se existir valor na coluna 'hub_suburb' mantém o valor. Senão, considere 'hub_city'
hub_geodata_df["hub_suburb"] = np.where(hub_geodata_df["hub_suburb"].notna(), hub_geodata_df["hub_suburb"], hub_geodata_df["hub_city"])
# Remoção da coluna 'hub_town' por já ter conseguido os valores do bairro e cidade
hub_geodata_df = hub_geodata_df.drop("hub_town", axis=1)
hub_geodata_df.head()

In [ ]:
# Junta o hub_df com hub_geodata_df, mantendo apenas as coluans de interesse
hub_df = pd.merge(left=hub_df, right=hub_geodata_df, left_index=True, right_index=True)
hub_df = hub_df[["region", "hub_suburb", "hub_city"]]
# Agora juntar o deliveries_df com o hub_df, com base na coluna 'region' a relação e redefinindo a ordem das colunas
deliveries_df = pd.merge(left=deliveries_df, right=hub_df, how="inner", on="region")
deliveries_df = deliveries_df[["name", "region", "hub_lng", "hub_lat", "hub_city", "hub_suburb", "vehicle_capacity", "delivery_size", "delivery_lng", "delivery_lat"]]
deliveries_df.head()

Como existem apenas 3 geolocalizações distintas no hubs no dataframe, porém com 636149 distintas para as entregas, demoraria muito para fazer a consulta pelo Nominatim. Por isso, será usado um link com o resultado para facilitar a análise.

In [ ]:
wget.download("https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries-geodata.csv")
deliveries_geodata_df = pd.read_csv('deliveries-geodata.csv')
deliveries_geodata_df.head()

In [ ]:
# Inclusão do deliveries_geodata_df no deliveries_df
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_geodata_df[["delivery_city", "delivery_suburb"]], how="inner", left_index=True, right_index=True)
deliveries_df.head()

Sobre a estrutura do deliveries_df depois de toda a atualização:

In [ ]:
len(deliveries_df)

In [ ]:
# Aqui é possivel ver que há valores nulos em deliveries_df["delivery_city"] e deliveries_df["delivery_suburb"] 
deliveries_df.info()

In [ ]:
deliveries_df.isna().any()

In [ ]:
100 * (deliveries_df["delivery_city"].isna().sum() / len(deliveries_df))

In [ ]:
100 * (deliveries_df["delivery_suburb"].isna().sum() / len(deliveries_df))

In [ ]:
prop_df = deliveries_df[["delivery_suburb"]].value_counts() / len(deliveries_df)
prop_df.sort_values(ascending=False).head(10)

## 5\. Visualização

In [ ]:
# Extração do mapa do distrito federal
wget.download("https://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc100/go_df/versao2016/shapefile/bc100_go_df_shp.zip")
with zipfile.ZipFile('./bc100_go_df_shp.zip', 'r') as zip:
    zip.extractall('./all')

In [ ]:
# Transformação do mapa em uma serie datapandas
mapa = geopandas.read_file("./all/LIM_Unidade_Federacao_A.shp")
mapa = mapa.loc[[0]]
mapa.head()

In [ ]:
# Definindo a localização dos 3 hubs da Loggi no eixo x e y do gráfico em GeoDataFrame
hub_df = deliveries_df[["region", "hub_lng", "hub_lat"]].drop_duplicates().reset_index(drop=True)
geo_hub_df = geopandas.GeoDataFrame(hub_df, geometry=geopandas.points_from_xy(hub_df["hub_lng"], hub_df["hub_lat"]))
geo_hub_df.head()

In [ ]:
# Definindo a localização de cada entrega dessa instancia da Loggi no eixo x e y do gráfico em GeoDataFrame
geo_deliveries_df = geopandas.GeoDataFrame(deliveries_df, geometry=geopandas.points_from_xy(deliveries_df["delivery_lng"], deliveries_df["delivery_lat"]))
geo_deliveries_df.head()

In [ ]:
# cria o plot vazio
fig, ax = plt.subplots(figsize = (40/2.54, 40/2.54))

# plot mapa do distrito federal
mapa.plot(ax=ax, alpha=0.4, color="lightgrey")

# plot das entregas
geo_deliveries_df.query("region == 'df-0'").plot(ax=ax, markersize=1, color="red", label="df-0")
geo_deliveries_df.query("region == 'df-1'").plot(ax=ax, markersize=1, color="blue", label="df-1")
geo_deliveries_df.query("region == 'df-2'").plot(ax=ax, markersize=1, color="seagreen", label="df-2")

# plot dos hubs
geo_hub_df.plot(ax=ax, markersize=30, marker="x", color="black", label="hub")

# plot da legenda
plt.title("Entregas no Distrito Federal por Região", fontdict={"fontsize": 16})
lgnd = plt.legend(prop={"size": 15})
for handle in lgnd.legend_handles:
    handle.set_sizes([50])

Gerei também outros 5 gráficos. O primeiro é sobre a proporção de entregas por região, assim como o mapa demonstra. Os 3 são sobre valores como a quantidade média, minima e máxima de entregas por instância em cada região. O último é sobre qual cidade possui o maior indice de entregas.

In [ ]:
# Gráfico de proporção de entregas por região

data = pd.DataFrame(deliveries_df[['region', 'vehicle_capacity']].value_counts(normalize=True)).reset_index()
data.rename(columns={'proportion': "region_percent", 0 : "region_percent"}, inplace=True)

cores = {'df-0': 'red', 'df-1': 'blue', 'df-2': 'green'}

with sns.axes_style('whitegrid'):
  grafico = sns.barplot(data=data, x="region", y="region_percent", errorbar=None, palette=cores, color=cores.values(),width=0.5)
  grafico.set(title='Proporção de entregas por região', xlabel='Regiões', ylabel='Proporção')

In [ ]:
geo_deliveries_df = pd.DataFrame(geo_deliveries_df)

df_region0 = geo_deliveries_df.loc[geo_deliveries_df['region'] == 'df-0']
teste_df = pd.DataFrame(df_region0[['name','geometry','region']].groupby('name').count().sort_values(by='geometry').reset_index())
teste_df = teste_df.describe().T.rename(columns = {'mean':'média'})
teste_df = teste_df[['min','média','max',]].T.reset_index().rename(columns = {'index':'valores','geometry':'entregas'})

with sns.axes_style('whitegrid'):
  grafico = sns.barplot(data=teste_df, x="valores", y="entregas",errorbar=None,width=0.5,palette='Reds')
  grafico.set(title='Quantidade de entregas por instância: DF-0', xlabel=None, ylabel='Nº de entregas');


In [ ]:
df_region1 = geo_deliveries_df.loc[geo_deliveries_df['region'] == 'df-1']
teste_df1 = pd.DataFrame(df_region1[['name','geometry','region']].groupby('name').count().sort_values(by='geometry').reset_index())
teste_df1 = teste_df1.describe().T.rename(columns = {'mean':'média'})
teste_df1 = teste_df1[['min','média','max',]].T.reset_index().rename(columns = {'index':'valores','geometry':'entregas'})

with sns.axes_style('whitegrid'):
  grafico = sns.barplot(data=teste_df1, x="valores", y="entregas", errorbar=None,width=0.5,palette='Blues')
  grafico.set(title='Quantidade de entregas por instância: DF-1', xlabel=None, ylabel='Nº de entregas');

In [ ]:
df_region2 = geo_deliveries_df.loc[geo_deliveries_df['region'] == 'df-2']
teste_df2 = pd.DataFrame(df_region2[['name','geometry','region']].groupby('name').count().sort_values(by='geometry').reset_index())
teste_df2 = teste_df2.describe().T.rename(columns = {'mean':'média'})
teste_df2 = teste_df2[['min','média','max',]].T.reset_index().rename(columns = {'index':'valores','geometry':'entregas'})

with sns.axes_style('whitegrid'):
  grafico = sns.barplot(data=teste_df2, x="valores", y="entregas", errorbar=None,width=0.5,palette='Greens')
  grafico.set(title='Quantidade de entregas por instância: DF-2', xlabel=None, ylabel='Nº de entregas');

In [ ]:
cidades_df = deliveries_df[['delivery_city']]
cidades_df['quantidade'] = 1
cidades_df = cidades_df.groupby('delivery_city').agg(sum).sort_values('quantidade', ascending=False).reset_index()
top_cidades_df = top_cidades_df.head(10)


with sns.axes_style('whitegrid'):
  plt.figure(figsize=(15,5))
  plt.grid(True)
  grafico = sns.barplot(data=top_cidades_df, x="delivery_city", y="quantidade", errorbar=None,width=0.5)
  grafico.set(title='Top 10 - Quantidade de entrega por cidades', xlabel=None, ylabel='Quantidade de entregas')

## 6\. Resultado


Pelo que é possivel ver pelos gráficos criados, a região DF-0 é onde tem menos entregas no geral. Isso se deve a questão topográfica da própria região de brasilia, pois a região DF-0 é onde a densidade populacional é bem menor comparada as outras regiões. Enquanto que a região DF-1 é a com maior número de entregas feitas, muito provavelmente por ser a região central.

Outro ponto a ser destacado é sobre as entregas que estão muito mais focadas em Brasilia comparado as outras cidades na região, demonstrando um acumulo de interesse na empresa na area mais central e que possivelmente o serviço de entrega seja melhro por isso.

Também é possivel perceber que os dados da Loggi possuem poucos valores nulos no geral, o que não chegou a atrapalhar a análise.